In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import optimize
from scipy.stats import poisson
import math

In [26]:
#import data from excel
data = pd.read_excel(r'C:\Users\clair\OneDrive\Desktop\sup_junc_data.xlsx', dtype={'Name': str, 'Value': float})
#rC':\Users\clair\OneDrive\Desktop\Schewl\Yale Stuff\Junior Year\AdvancedLab\'

In [27]:
data

,NIN_1_1_inc_I,NIN_1_1_inc_V,NIN_1_1_inc_V_err,NIN_1_1_dec_I,NIN_1_1_dec_V,NIN_1_1_dec_V_err,SIN_1_1_inc_I,SIN_1_1_inc_V,SIN_1_1_inc_V_err,SIN_1_1_dec_I,SIN_1_1_dec_V,SIN_1_1_dec_V_err,SIS_1_1_inc_I,SIS_1_1_inc_V,SIS_1_1_inc_V_err,SIS_1_1_dec_I,SIS_1_1_dec_V,SIS_1_1_dec_V_err
0,-0.00050,-0.007325,0.000015,-0.00050,-0.007327,0.000028,-0.00050,-0.009033,0.000000,-0.00050,-0.009031,0.000020,-0.00002,-0.001388,0.000003,-0.00002,-0.001396,0.000003
1,-0.00049,-0.007308,0.000061,-0.00049,-0.007324,0.000000,-0.00049,-0.008792,0.000027,-0.00049,-0.008790,0.000013,-0.00002,-0.001387,0.000003,-0.00002,-0.001395,0.000003
2,-0.00048,-0.007080,0.000000,-0.00048,-0.007085,0.000036,-0.00048,-0.008729,0.000105,-0.00048,-0.008683,0.000121,-0.00002,-0.001387,0.000002,-0.00002,-0.001394,0.000004
3,-0.00047,-0.007027,0.000101,-0.00047,-0.007059,0.000069,-0.00047,-0.008545,0.000000,-0.00047,-0.008545,0.000000,-0.00002,-0.001386,0.000002,-0.00002,-0.001393,0.000005
4,-0.00046,-0.006836,0.000000,-0.00046,-0.006838,0.000022,-0.00046,-0.008306,0.000037,-0.00046,-0.008304,0.000028,-0.00002,-0.001386,0.000003,-0.00002,-0.001391,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00002,0.000918,0.000002,0.00002,0.000918,0.000002
397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00002,0.000918,0.000002,0.00002,0.000918,0.000002
398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00002,0.000919,0.000003,0.00002,0.000919,0.000004
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00002,0.000920,0.000004,0.00002,0.000920,0.000004


In [22]:
#guesses for the fit parameters
R     = 0
delta = 0
T     = 0

firstGuess = [R, delta, T]

In [ ]:
def SINcurr(v, params=[]):
    # energy and temperature measured in voltage units
    #    1 meV = 1 mV
    #    1 K   = 0.0861 mV
    # Setting parameters
    
    r_0    = firstGuess[0]
    delta  = firstGuess[1]
    t      = firstGuess[2] 
    
    e_max  = 999 #cutoff for approximating integral
    e_step = 999 #step size for approximating integral
    r_par  = 999 #resistance parallel to junction
    
    e = np.arange(-e_max, e_max, e_step)
    [col, nptse] = len(e)
    
    #Import voltage data
    #Rather than taking this vector of uniformly spaced fake voltages,
    #you should import a vector containing your real voltage data.
    [col, nptsv]    = len(v)
    ds_pb, di, itot = np.zeros(len(nptsv))
    
    # step over voltage
    # for k=1:nptsv
    for k in range(len(nptsv)):
        vv = v[k]
        
        # do the integral over e
        for j in range(len(nptse)):
            if np.absolute(e[j]) > delta:
                # tame divergence in bcs density of states used .99 instead of 1
                ds_pb[j] = np.absolute(e[j])/np.sqrt((e[j])**2-.99*delta**2)
                
            else:
                ds_pb[j] = 0
                
            di[j] = ds_pb[j]*(1/(np.exp(e[j]/t)+1)-1/(np.exp((e[j]+vv)/t)+1))
            
        # integrate with trapezoid rule include a parallel resistance r_par
        itot[k] = np.trapz(di,e)/r_0 + vv/r_par
        
    return itot

In [11]:
#main method -- minimizing chi squared
def herbert(firstGuess = []):

    #original parameter guesses
    r    = firstGuess[0]
    delt = firstGuess[1]
    t    = firstGuess[2]
    
    #range over which to evaluate 
    R_range   = np.arange()
    del_range = np.arange()
    T_range   = np.arange()
    
    #final optimized values of fit parameters
    optim_R   = 0
    optim_del = 0
    optim_T   = 0
    
    #default maximum value
    scoreboard = 1000000000000000000000000
    
    for i in R_range:
        for j in del_range:
            for k in T_range:
                #calculate reduced chi squared
                sum_lsq = 0
                lsq = (SINcurr()**2)
                sum_lsq += lsq

                if(chi_sq < scoreboard):
                    print("surpassed scoreboard")
                    scoreboard = sum_lsq
                    optim_R    = R_range[i]
                    optim_del  = del_range[j]
                    optim_T    = T_range[k]
                        
    print("R: " + optim_R + "\ndelta: " + optim_del + "\nT: " + optim_T)   
    return optim_R, optim_del, optim_T
        
    

In [12]:
print(herbert(firstGuess=[R, delta, T]))

#plt.scatter()
#plt.plot(SINcurr)
#plt.plot(SINcurr)

NameError: name 'R' is not defined

In [332]:
params, params_covariance = optimize.curve_fit(N_global, beta, Nc, p0=[4000, 45, 45, 10, 40])
#print(params)
#p0=[500, 45, 45, 20, 0.1]) this got us close
#params = [4000, 45, 45, 10, 40]

In [67]:
def red_chisq(o, c, var, m):
    #v = n - m
    v = len(o) - m
    chi = 0
    for i in range(len(o)):
        chi = chi + ( (o[i] - c[i])**2) / (var[i]**2)
        #print(o[i]-c[i])
        #print(str(beta[i])+ ": "+str((o[i]-c[i])**2/var[i]**2))
        
    return chi / v